# OA Normalise (Silver OA Entity)
Create normalized OA entity records with extracted trial_id, data_stream_type, data_provider_name using LLM.

**Input**: `bronze_md.md_oa_file_raw`

**Output**: `silver_md.md_dta_operational_agreement_draft`


In [ ]:
# Cell 0: Imports and UDF

import json
import requests
from pyspark.sql import functions as F
from pyspark.sql.types import StructType, StructField, StringType, BooleanType
from pyspark.sql.window import Window

from clinical_data_standards_framework.utils import (
    save_with_audit,
    compute_definition_hash,
)

# -------------------------------------------------------------------
# UDF: OA definition hash (based on OA id + trial/data context)
# -------------------------------------------------------------------

@F.udf(StringType())
def oa_definition_hash_udf(
    operational_agreement_id: str,
    trial_id: str,
    data_stream_type: str,
    data_provider_name: str,
) -> str:
    """
    Compute a stable definition hash for OA rows.
    """
    record = {
        "operational_agreement_id": operational_agreement_id or "",
        "trial_id": trial_id or "",
        "data_stream_type": data_stream_type or "",
        "data_provider_name": data_provider_name or "",
    }
    return compute_definition_hash(record, list(record.keys()))

print("✅ Framework loaded successfully")


In [ ]:
# Cell 1: Read config & globals from setup task

globals_dict = json.loads(
    dbutils.jobs.taskValues.get(taskKey="setup", key="globals")
)
services_dict = json.loads(
    dbutils.jobs.taskValues.get(taskKey="setup", key="services")
)

try:
    pipeline_config = json.loads(
        dbutils.jobs.taskValues.get(taskKey="setup", key="pipeline_config")
    )
except Exception:
    pipeline_config = None

flow_name = dbutils.jobs.taskValues.get(taskKey="setup", key="flow_name")

print(f"Flow from setup: {flow_name}")
print("Globals from YAML:", globals_dict)
print("Services from YAML:", list(services_dict.keys()))

# Audit fields
created_by_principal = dbutils.jobs.taskValues.get(taskKey="setup", key="created_by_principal")
databricks_job_id    = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_id")
databricks_job_name  = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_job_name")
databricks_run_id    = dbutils.jobs.taskValues.get(taskKey="setup", key="databricks_run_id")


In [ ]:
# Cell 2: Resolve OA pipeline configuration

if not pipeline_config:
    raise ValueError(
        "pipeline_config task value is missing; "
        "ensure setup task ran with flow_name='operational_agreement_processor'"
    )

docs_cfg   = pipeline_config["documents"]["operational_agreement"]
output_cfg = docs_cfg["output"]

# From globals
catalog       = globals_dict["catalog"]        # e.g. "dta_poc"
bronze_schema = globals_dict["bronze_schema"]  # e.g. "bronze_md"
silver_schema = globals_dict["silver_schema"]  # e.g. "silver_md"

# From OA pipeline config
raw_table_name = output_cfg["raw_table_name"]  # e.g. "md_oa_file_raw"

raw_table_full    = f"{bronze_schema}.{raw_table_name}"
silver_table_full = f"{silver_schema}.md_dta_operational_agreement_draft"

print("\nResolved OA config (Silver stage):")
print(f"  catalog             = {catalog}")
print(f"  bronze_schema       = {bronze_schema}")
print(f"  silver_schema       = {silver_schema}")
print(f"  raw_table_full      = {raw_table_full}")
print(f"  silver_table_full   = {silver_table_full}")


In [ ]:
# Cell 3: Utility and Read OA raw table (bronze)

def use_catalog_and_schema(catalog: str, schema: str):
    spark.sql(f"USE CATALOG `{catalog}`")
    spark.sql(f"CREATE SCHEMA IF NOT EXISTS `{schema}`")
    spark.sql(f"USE SCHEMA `{schema}`")

use_catalog_and_schema(catalog, bronze_schema)

# Note: file_name, file_extension, file_version removed - available in md_file_history
df_raw = spark.table(raw_table_full).select(
    "document_id",
    "parent_document_id",
    "content",
)

raw_count = df_raw.count()
print(f"\n[nb_tsdta_operational_agreement] Rows in {raw_table_full}: {raw_count}")

if raw_count == 0:
    print("No OA raw rows to process; exiting.")
    dbutils.notebook.exit("No OA raw documents to normalise")

# Keep the latest per parent_document_id as representative OA document
w = Window.partitionBy("parent_document_id").orderBy(F.col("document_id").desc())

df_raw_rep = (
    df_raw
    .withColumn("rn", F.row_number().over(w))
    .where(F.col("rn") == 1)
    .drop("rn")
)

rep_count = df_raw_rep.count()
print(f"Distinct parent_document_id (latest version only): {rep_count}")


In [ ]:
# Cell 4: Model endpoint + prompt for document-level entities
# Service config provides connection details; pipeline config provides use-case prompts

# Get pipeline-specific LLM config
llm_pipeline_cfg = docs_cfg.get("llm", {})
if not llm_pipeline_cfg:
    raise ValueError(
        "LLM configuration not found in pipeline config. "
        "Ensure 'llm:' section exists under operational_agreement in clinical_data_standards.yaml"
    )

# Get the service name from pipeline config (defaults to gpt_entity_extractor)
service_name = llm_pipeline_cfg.get("service", "gpt_entity_extractor")

# Get generic service connection details
llm_service = services_dict.get(service_name)
if not llm_service:
    raise ValueError(
        f"Service '{service_name}' not found in services config. "
        "Ensure clinical_data_standards.yaml has this service defined and config cache is refreshed."
    )

# Connection details from SERVICE (generic, reusable)
url = llm_service["endpoint"]
MODEL_NAME = llm_service["model"]
TIMEOUT_SECONDS = llm_service.get("timeout_seconds", 120)

# Get API key from secrets (scope and key from service config)
api_key = dbutils.secrets.get(
    llm_service["secret_scope"],
    llm_service["secret_key"]
)

headers = {
    "api-key": api_key,
    "Content-Type": "application/json",
}

# Prompts from PIPELINE config (use-case specific)
system_prompt = llm_pipeline_cfg.get("system_prompt", "You are a helpful assistant.")
doc_prompt_header = llm_pipeline_cfg["prompt"]["header"].strip()
doc_prompt_footer = llm_pipeline_cfg["prompt"]["footer"].strip()

print("✅ LLM configured:")
print(f"   Service: {service_name}")
print(f"   Endpoint: {url}")
print(f"   Model: {MODEL_NAME}")
print(f"   Timeout: {TIMEOUT_SECONDS}s")
print(f"   System prompt: {system_prompt[:50]}...")


In [ ]:
# Cell 5: Entity extraction function and UDF

def extract_doc_entities(doc_text: str) -> str:
    """
    Call the GPT endpoint once per document and return a CLEAN JSON string
    with keys: trial_id, data_stream_type, data_provider_name.
    """
    # If no content, just return empty JSON
    if doc_text is None or doc_text.strip() == "":
        return json.dumps({
            "trial_id": "",
            "data_stream_type": "",
            "data_provider_name": ""
        })

    full_prompt = f"{doc_prompt_header}\n{doc_text}\n{doc_prompt_footer}"

    data = {
        "model": MODEL_NAME,
        "messages": [
            {
                "role": "system",
                "content": system_prompt,
            },
            {
                "role": "user",
                "content": full_prompt,
            },
        ],
    }

    try:
        resp = requests.post(url, headers=headers, json=data, timeout=TIMEOUT_SECONDS)
        resp.raise_for_status()
        resp_json = resp.json()
        raw_content = resp_json["choices"][0]["message"]["content"]
    except Exception as e:
        print(f"⚠ Error calling OA entity model: {e}")
        return json.dumps({
            "trial_id": "",
            "data_stream_type": "",
            "data_provider_name": ""
        })

    if not raw_content:
        return json.dumps({
            "trial_id": "",
            "data_stream_type": "",
            "data_provider_name": ""
        })

    # --- Clean the model output down to pure JSON ---
    text = raw_content.strip()

    # If the model wrapped the JSON in ```json ... ``` or ``` ... ```
    start = text.find("{")
    end = text.rfind("}")

    if start != -1 and end != -1 and end > start:
        json_candidate = text[start : end + 1]
    else:
        print("⚠ Could not find JSON braces in model output; falling back to empty JSON.")
        return json.dumps({
            "trial_id": "",
            "data_stream_type": "",
            "data_provider_name": ""
        })

    # Validate that json_candidate is valid JSON; if not, fall back
    try:
        parsed = json.loads(json_candidate)
        cleaned = {
            "trial_id": parsed.get("trial_id", "") or "",
            "data_stream_type": parsed.get("data_stream_type", "") or "",
            "data_provider_name": parsed.get("data_provider_name", "") or "",
        }
        return json.dumps(cleaned)
    except Exception as e:
        print(f"⚠ Failed to parse JSON from model output: {e}")
        return json.dumps({
            "trial_id": "",
            "data_stream_type": "",
            "data_provider_name": ""
        })


@F.udf(returnType=StringType())
def doc_entities_json_udf(doc_text: str) -> str:
    return extract_doc_entities(doc_text)

print("✅ Entity extraction UDF defined")


In [ ]:
# Cell 6: Call the model and parse JSON into columns
# Note: file_name, file_extension, file_version removed - available in md_file_history

docs_with_json_df = df_raw.select(
    "document_id",
    "parent_document_id",
    "content",
    doc_entities_json_udf(F.col("content")).alias("entities_json"),
)

entities_schema = StructType([
    StructField("trial_id",           StringType(), True),
    StructField("data_stream_type",        StringType(), True),
    StructField("data_provider_name", StringType(), True),
])

parsed_entities_df = docs_with_json_df.select(
    "document_id",
    "parent_document_id",
    "content",
    "entities_json",
    F.from_json("entities_json", entities_schema).alias("entities"),
)

oa_entities_flat_df = parsed_entities_df.select(
    "document_id",
    "parent_document_id",
    # Flatten the struct
    F.col("entities.trial_id").alias("trial_id"),
    F.col("entities.data_stream_type").alias("data_stream_type"),
    F.col("entities.data_provider_name").alias("data_provider_name"),
)

print("\nSample of extracted entities (JSON):")
docs_with_json_df.select("document_id", "entities_json").show(5, truncate=False)

print("\nSample of flattened OA entities:")
oa_entities_flat_df.show(5, truncate=False)


In [ ]:
# Cell 7: Build Silver OA dataframe with DTA / versioning columns
# Note: file columns removed, version_tag renamed to version (consistent with TV/TC)

use_catalog_and_schema(catalog, silver_schema)

silver_table_full = f"{catalog}.{silver_schema}.md_dta_operational_agreement_draft"

# If the table already exists, reuse IDs and versioning where possible
if spark.catalog.tableExists(silver_table_full):
    existing_df = (
        spark.table(silver_table_full)
        .select(
            "document_id",
            "operational_agreement_id",
            "dta_id",
            "version",
            "version_status",
            "is_current_draft",
            "row_status",
            "vendor_comments",
        )
    )

    merged_df = (
        oa_entities_flat_df.alias("new")
        .join(existing_df.alias("old"), on="document_id", how="left")
        .select(
            # reuse existing OA id if present, else generate
            F.coalesce(F.col("old.operational_agreement_id"), F.expr("uuid()")).alias("operational_agreement_id"),
            F.col("new.document_id"),
            F.col("new.parent_document_id"),
            F.col("new.trial_id"),
            F.col("new.data_stream_type"),
            F.col("new.data_provider_name"),
            F.coalesce(F.col("old.dta_id"), F.lit(None).cast("string")).alias("dta_id"),
            F.coalesce(F.col("old.version"), F.lit(None).cast("string")).alias("version"),
            F.coalesce(F.col("old.version_status"), F.lit("DRAFT")).alias("version_status"),
            F.coalesce(F.col("old.is_current_draft"), F.lit(True)).alias("is_current_draft"),
            F.coalesce(F.col("old.row_status"), F.lit("COMPLETED")).alias("row_status"),
            F.coalesce(F.col("old.vendor_comments"), F.lit(None).cast("string")).alias("vendor_comments"),
        )
    )
else:
    merged_df = (
        oa_entities_flat_df
        .withColumn("operational_agreement_id", F.expr("uuid()"))
        .withColumn("dta_id",         F.lit(None).cast("string"))
        .withColumn("version",        F.lit(None).cast("string"))
        .withColumn("version_status", F.lit("DRAFT"))
        .withColumn("is_current_draft", F.lit(True))
        .withColumn("row_status",     F.lit("COMPLETED"))
        .withColumn("vendor_comments", F.lit(None).cast("string"))
    )

# Add definition_hash based on OA identity
oa_silver_df = merged_df.withColumn(
    "definition_hash",
    oa_definition_hash_udf(
        F.col("operational_agreement_id"),
        F.col("trial_id"),
        F.col("data_stream_type"),
        F.col("data_provider_name"),
    )
)

print("\nSample Silver OA rows (before write):")
oa_silver_df.select(
    "document_id",
    "trial_id",
    "data_stream_type",
    "data_provider_name",
    "operational_agreement_id",
    "dta_id",
    "version",
    "version_status",
    "is_current_draft",
).show(5, truncate=False)

row_count = oa_silver_df.count()
print(f"\n[nb_tsdta_operational_agreement] Rows to write to {silver_table_full}: {row_count}")

if row_count == 0:
    dbutils.notebook.exit("No OA entities extracted; nothing to write")


In [ ]:
# Cell 8: Write to Silver table with save_with_audit

save_with_audit(
    df=oa_silver_df,
    table_name=silver_table_full,
    created_by_principal=created_by_principal,
    databricks_job_id=databricks_job_id,
    databricks_job_name=databricks_job_name,
    databricks_run_id=databricks_run_id,
    mode="append",
    table_description=(
        "Normalized operational agreement documents with extracted trial_id, data_stream_type, "
        "data_provider_name, file metadata, document content, and DTA/versioning metadata."
    ),
)

print(
    f"\n✅ Successfully wrote {row_count} row(s) to "
    f"{silver_table_full} with save_with_audit (mode=append)"
)

# Set task values for downstream notebooks
dbutils.jobs.taskValues.set(key="oa_normalise_status", value="SUCCESS")
dbutils.jobs.taskValues.set(key="oa_normalise_count", value=str(row_count))
